In [1]:
from PyQt5 import QtCore,QtGui,QtWidgets
import sys,math,os
import qtawesome
from time import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import numpy as np
import pandas as pd
import datetime
import matplotlib
matplotlib.use("Qt5Agg")  # 声明使用QT5
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import pyqtgraph as pg 
import pyqtgraph.exporters as pe
import qdarkstyle, requests, sys, time, random, json, datetime, re

In [2]:
# 引入常用库
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import datetime as dt
import itertools # 迭代器工具
# import talib # 技术分析
import math
# 画图
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
# import seaborn as sns
from pylab import * 
import matplotlib
# 设置字体 用来正常显示中文标签
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']
# 用来正常显示负号
plt.rcParams['axes.unicode_minus'] = False
# 图表主题
plt.style.use('ggplot')
# 忽略报错
import warnings
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
def cal_statistics(df):
    # 净值序列
    df['net_asset_pct_chg'] = df.net_asset_value.pct_change(1).fillna(0)

    # 总收益率和年化收益率
    total_ret = df['net_asset_value'].iloc[-1]-1
    annual_ret = total_ret**(1/(df.shape[0]/252))-1
    total_ret = total_ret*100
    annual_ret = annual_ret*100

    # 夏普比率
    sharp_ratio = df['net_asset_pct_chg'].mean() * math.sqrt(252)/df['net_asset_pct_chg'].std()

    # 回撤
    df['high_level'] = df['net_asset_value'].rolling(min_periods = 1, window = len(df),center = False).max()
    df['draw_down'] = df['net_asset_value'] - df['high_level']
    df['draw_down_pct'] = df['draw_down'] / df['high_level'] * 100
    max_draw_down = df['draw_down'].min()
    max_draw_down_pct = df['draw_down_pct'].min()

    # 持仓总天数
    hold_days = df['position'].sum()

    # 交易次数
    trade_count = df[df['flag']!=0].shape[0]/2

    # 平均持仓天数
    avg_hold_days = int(hold_days/trade_count)

    # 获利天数
    profit_days = df[df['net_asset_pct_chg']>0].shape[0]
    # 亏损天数
    loss_days = df[df['net_asset_pct_chg']<0].shape[0]

    # 胜率（按天）
    winrate_day = profit_days/(profit_days+loss_days)*100
    # 平均盈利率（按天）
    avg_profit_rate_day = df[df['net_asset_pct_chg']>0]['net_asset_pct_chg'].mean()*100
    # 平均亏损率（按天）
    avg_loss_rate_day = df[df['net_asset_pct_chg']<0]['net_asset_pct_chg'].mean()*100
    # 平均盈亏比（按天）
    avg_pl_ratio_day = avg_profit_rate_day/ abs(avg_loss_rate_day)

    # 每次交易情况
    buy_trades = df[df['flag']==1].reset_index()
    sell_trades = df[df['flag']==-1].reset_index()
    result_trade = {
        'buy':buy_trades['close'],
        'sell':sell_trades['close'],
        'pct_chg':(sell_trades['close'] - buy_trades['close'])/buy_trades['close']
    }
    result_trade = pd.DataFrame(result_trade)

    # 盈利次数
    profit_trades = result_trade[result_trade['pct_chg']>0].shape[0]
    # 亏损次数
    loss_trades = result_trade[result_trade['pct_chg']<0].shape[0]
    # 单次最大盈利
    max_profit_trade = result_trade['pct_chg'].max()*100
    # 单次最大亏损
    max_loss_trade = result_trade['pct_chg'].min()*100
    # 胜率（按次）
    winrate_trade = profit_trades/(profit_trades+loss_trades)*100
    # 平均盈利率（按次）
    avg_profit_rate_trade = result_trade[result_trade['pct_chg']>0]['pct_chg'].mean()*100
    # 平均亏损率（按次）
    avg_loss_rate_trade = result_trade[result_trade['pct_chg']<0]['pct_chg'].mean()*100
    # 平均盈亏比（按次）
    avg_pl_ratio_trade = avg_profit_rate_trade/ abs(avg_loss_rate_trade)

    statistics_result = {
        'net_asset_value':df['net_asset_value'].iloc[-1],#最终净值
        'total_return':total_ret,#收益率
        'annual_return':annual_ret,#年化收益率
        'sharp_ratio':sharp_ratio,#夏普比率
        'max_draw_percent':max_draw_down_pct,#最大回撤
        'hold_days':hold_days,#持仓天数
        'trade_count':trade_count,#交易次数
        'avg_hold_days':avg_hold_days,#平均持仓天数
        'profit_days':profit_days,#盈利天数
        'loss_days':loss_days,#亏损天数
        'winrate_by_day':winrate_day,#胜率（按天）
        'avg_profit_rate_day':avg_profit_rate_day,#平均盈利率（按天）
        'avg_loss_rate_day':avg_loss_rate_day,#平均亏损率（按天）
        'avg_profit_loss_ratio_day':avg_pl_ratio_day,#平均盈亏比（按天）
        'profit_trades':profit_trades,#盈利次数
        'loss_trades':loss_trades,#亏损次数
        'max_profit_trade':max_profit_trade,#单次最大盈利
        'max_loss_trade':max_loss_trade,#单次最大亏损
        'winrate_by_trade':winrate_trade,#胜率（按次）
        'avg_profit_rate_trade':avg_profit_rate_trade,#平均盈利率（按次）
        'avg_loss_rate_trade':avg_loss_rate_trade,#平均亏损率（按次）
        'avg_profit_loss_ratio_trade':avg_pl_ratio_trade#平均盈亏比（按次）
    }

    return statistics_result

In [4]:
class Trade():
    def __init__(self, invest_cash):
        #申明参数
        self.signal_date = None  
        self.trade_price = None
        self.signal = None

        # 参数：初始资金 1
        self.investcash = invest_cash
        self.commission = 0.001

        # 交易数据
        self.cash = invest_cash  #初始化现金为上期现金值 
        self.volumn = 0 #初始化成交量，全部成交
        self.position_value = 0 #初始化持仓价值
        self.all_value = invest_cash #初始化资产价值
        self.cost=0 #初始化交易成本
        self.position=0 #初始化持有头寸

    def buy(self):
        self.position=self.volumn
        long_position_value = self.position * self.trade_price #考虑佣金后，持仓价值与初始投资不同        
        self.cost = self.commission * long_position_value
        
        self.cash = 0 #所有现金转为0，全部买入标的
        self.position_value = self.cash + long_position_value #所有头寸今天的价值
        self.all_value=self.cash+self.position_value

    def sell(self): 
        short_position_value = self.volumn * self.trade_price # 将卖出的头寸价值，为负数

        self.cost = self.commission * abs(short_position_value)
        self.cash = self.cash - short_position_value - self.cost #头寸全部卖出
        
        self.position +=self.volumn  #所有头寸全部卖出
        self.position_value = 0
        self.all_value = self.cash + self.position_value

    def remain(self):
        self.get_position_value()
        self.value = self.cash + self.position_value

    def get_position_value(self):
        self.position_value = self.position * self.trade_price

    def signalcoming(self, date, price, signal, number):
        self.signal_date = date
        self.trade_price = price  
        self.signal = signal
        self.volumn = number

    def trade(self):
        """ 开始交易"""
        if self.signal > 0:
            if self.cash > self.trade_price*self.volumn: #需有足够资金才能买入
                self.buy()
        elif self.signal < 0:  
            if self.position > 0:  #持有头寸才能生效       
                self.sell()
        elif self.signal == 0:
            self.remain()
        else:
            raise ValueError('Invalid "signal" value!')
            
        self.value = self.position_value + self.cash   
        trade_info = self.get_trade_data()
        
        self.show_trading_info()  # 打印交易详情
        
        return trade_info

    # 获取并存储交易价格，交易信号，资产价值，成交量，头寸价值，现金账户和成本7个参数数据
    def get_trade_data(self):
        parameter_list = ['trade_price', 'signal', 'all_value', 'position', 'position_value', 'cash', 'cost']
        value_list = {name: getattr(self, name) for name in parameter_list}
        return value_list
    
    def show_trading_info(self):
        global result
        
        if self.signal > 0:
            a=f'{self.signal_date} 买入  ，'f'当前持仓价值 {self.position_value:.2f}，总现金价值 {self.cash:.2f} '
            #print(a)
            result.append(a)
        elif self.signal < 0:
            b=f'{self.signal_date} 卖出  ，'f'当前持仓价值 {self.position_value:.2f}，总现金价值 {self.cash:.2f} '
            #print(b)
            result.append(b)
        else:
            c=f'{self.signal_date} 无操作，当前持仓价值 {self.position_value:.2f}，总资产价值 {self.value:.2f}'
            #print(c)
            result.append(c)

In [5]:
class MainUi(QtWidgets.QMainWindow):
    
    def __init__(self):
        super().__init__()
        self.setWindowTitle("算法交易执行系统")
        self.init_ui()
        self.th = MyThread()
        self.th.signalForText.connect(self.onUpdateText)
        sys.stdout = self.th
        

    def onUpdateText(self,text):
        cursor = self.process.textCursor()
        cursor.movePosition(QTextCursor.End)
        cursor.insertText(text)
        self.process.setTextCursor(cursor)
        self.process.ensureCursorVisible()

    def init_ui(self):
        self.setFixedSize(1200,900)
        self.main_widget = QtWidgets.QWidget() # 创建窗口主部件
        self.main_layout = QtWidgets.QGridLayout() # 创建主部件的网格布局
        self.main_widget.setLayout(self.main_layout) # 设置窗口主部件布局为网格布局

        self.left_widget = QtWidgets.QWidget() # 创建左侧部件
        self.left_widget.setObjectName('left_widget')
        self.left_layout = QtWidgets.QGridLayout() # 创建左侧部件的网格布局层
        self.left_widget.setLayout(self.left_layout) # 设置左侧部件布局为网格

        self.right_widget = QtWidgets.QWidget() # 创建右侧部件
        self.right_widget.setObjectName('right_widget')
        self.right_layout = QtWidgets.QGridLayout()
        self.right_widget.setLayout(self.right_layout) # 设置右侧部件布局为网格

        self.main_layout.addWidget(self.left_widget,0,0,1,1) # 左侧部件在第0行第0列，占8行3列
        self.main_layout.addWidget(self.right_widget,0,2,1,1) # 右侧部件在第0行第3列，占8行9列
        self.setCentralWidget(self.main_widget) # 设置窗口主部件

        #####################添加
        self.left_close = QtWidgets.QPushButton("") # 关闭按钮
        self.left_visit = QtWidgets.QPushButton("") # 空白按钮
        self.left_mini = QtWidgets.QPushButton("") # 最小化按钮

        #交易品种
        self.recommend_label = QtWidgets.QLabel("交易品种")
        self.recommend_label.setObjectName('lable')
        self.left_layout.addWidget(self.recommend_label, 0, 0, 1, 1)
        self.recommend_combo = QComboBox(self)
        self.recommend_combo.addItem("000001.XSHE")
        self.left_layout.addWidget(self.recommend_combo, 0, 1, 1, 1)

        #交易时间
        self.recommend_label = QtWidgets.QLabel("交易开始时间")
        self.recommend_label.setObjectName('lable')
        self.left_layout.addWidget(self.recommend_label, 1, 0, 1, 1)
        self.recommend_combo = QComboBox(self)
        self.recommend_combo.addItem("2023-03-14")
        self.left_layout.addWidget(self.recommend_combo, 1, 1, 1, 1)

        self.recommend_label = QtWidgets.QLabel("交易结束时间")
        self.recommend_label.setObjectName('lable')
        self.left_layout.addWidget(self.recommend_label, 2, 0, 1, 1)
        self.recommend_combo = QComboBox(self)
        self.recommend_combo.addItem("2023-04-06")
        self.left_layout.addWidget(self.recommend_combo, 2, 1, 1, 1)

        #数据频率
        self.recommend_label = QtWidgets.QLabel("数据频率")
        self.recommend_label.setObjectName('lable')
        self.left_layout.addWidget(self.recommend_label, 3, 0, 1, 1)
        self.recommend_combo = QComboBox(self)
        self.recommend_combo.addItem("1min")
        self.left_layout.addWidget(self.recommend_combo, 3, 1, 1, 1)

        #账户初始金额
        self.recommend_label = QtWidgets.QLabel("账户初始金额")
        self.recommend_label.setObjectName('lable')
        self.left_layout.addWidget(self.recommend_label, 4, 0, 1, 1)
        self.recommend_combo = QComboBox(self)
        self.recommend_combo.addItem("100")
        self.left_layout.addWidget(self.recommend_combo, 4, 1, 1, 1)
        self.recommend_label_1 = QtWidgets.QLabel("(单位:万元)")
        self.left_layout.addWidget(self.recommend_label_1, 4, 2, 1, 1)
        
        #输出监控
        self.process = QTextEdit(self, readOnly=True)
        self.process.ensureCursorVisible()
        self.process.setLineWrapColumnOrWidth(800)
        self.process.setLineWrapMode(QTextEdit.FixedPixelWidth)
        self.process.setFixedWidth(1300)
        self.process.setFixedHeight(600)
        self.process.move(30, 50)
        self.right_layout.addWidget(self.process, 2, 2, 1, 1)
        
        #执行按钮
        self.search_button = QtWidgets.QPushButton(qtawesome.icon('fa.home', color='red'), "执行交易")
        self.search_button.clicked.connect(self.genMastClicked)
        self.search_button.setObjectName('button')
        self.search_button.setFont(qtawesome.font('fa', 16))
        self.right_layout.addWidget(self.search_button, 4, 2, 1, 1)
    
        #查看回测
        self.search_button_1 = QtWidgets.QPushButton(qtawesome.icon('fa.home', color='red'), "查看回测指标")
        self.search_button_1.setObjectName('button1')
        self.search_button_1.setFont(qtawesome.font('fa', 16))
        self.right_layout.addWidget(self.search_button_1, 4, 3, 1, 1)
        
        #查看实际交易图
        self.search_button_2 = QtWidgets.QPushButton(qtawesome.icon('fa.home', color='red'), "查看触发信号")
        self.search_button_2.setObjectName('button2')
        self.search_button_2.setFont(qtawesome.font('fa', 16))
        self.right_layout.addWidget(self.search_button_2, 4, 4, 1, 1)
        
        #查看实际交易图
        self.search_button_3 = QtWidgets.QPushButton(qtawesome.icon('fa.home', color='red'), "查看净值曲线")
        self.search_button_3.setObjectName('button3')
        self.search_button_3.setFont(qtawesome.font('fa', 16))
        self.right_layout.addWidget(self.search_button_3, 4, 5, 1, 1)

    def search(self):
        try:
            self.t = MyThread()
            self.t.start()
        except Exception as e:
            raise e

    def genMastClicked(self):
        print('Running...')
        self.search()
        loop = QEventLoop()
        QTimer.singleShot(2000, loop.quit)
        loop.exec_()

    def closeEvent(self, event):
        """Shuts down application on close."""
        # Return stdout to defaults.
        sys.stdout = sys.__stdout__
        super().closeEvent(event)
            

In [6]:
class PdTable(QAbstractTableModel):
    def __init__(self, data):
        QAbstractTableModel.__init__(self)
        self._data = data
 
    def rowCount(self, parent=None):
        return self._data.shape[0]
 
    def columnCount(self, parent=None):
        return self._data.shape[1]
 
    # 显示数据
    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return str(self._data.iloc[index.row(), index.column()])
        return None
 
    # 显示行和列头
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return self._data.columns[col]
        elif orientation == Qt.Vertical and role == Qt.DisplayRole:
            return self._data.axes[0][col]
        return None

In [7]:
class MyFigure(FigureCanvas):
    def __init__(self,width=5, height=4, dpi=100):
        #第一步：创建一个创建Figure
        self.fig = Figure(figsize=(width, height), dpi=dpi)
        #第二步：在父类中激活Figure窗口
        super(MyFigure,self).__init__(self.fig) #此句必不可少，否则不能显示图形
        #第三步：创建一个子图，用于绘制图形用，111表示子图编号，如matlab的subplot(1,1,1)
        self.axes = self.fig.add_subplot(111)

class QPixmapDemo(QMainWindow):
    def __init__(self):
        super(QPixmapDemo, self).__init__()

        #设置窗口大小
        self.resize(600, 400)
        #设置窗口标题
        self.setWindowTitle("交易触发信号")

        pixmap = QPixmap("触发信号.png")
        label = QLabel(self)
        label.setPixmap(pixmap)

        #创建水平布局
        layout = QHBoxLayout()
        layout.addWidget(label)

        mainFrame = QWidget()
        mainFrame.setLayout(layout)
        self.setCentralWidget(mainFrame)

In [8]:
class MyFigure1(FigureCanvas):
    def __init__(self,width=5, height=4, dpi=100):
        #第一步：创建一个创建Figure
        self.fig = Figure(figsize=(width, height), dpi=dpi)
        #第二步：在父类中激活Figure窗口
        super(MyFigure1,self).__init__(self.fig) #此句必不可少，否则不能显示图形
        #第三步：创建一个子图，用于绘制图形用，111表示子图编号，如matlab的subplot(1,1,1)
        self.axes = self.fig.add_subplot(111)

class QPixmapDemo1(QMainWindow):
    def __init__(self):
        super(QPixmapDemo1, self).__init__()

        #设置窗口大小
        self.resize(800, 500)
        #设置窗口标题
        self.setWindowTitle("收益曲线")

        pixmap = QPixmap("111回测.png")
        label = QLabel(self)
        label.setPixmap(pixmap)

        #创建水平布局
        layout = QHBoxLayout()
        layout.addWidget(label)

        mainFrame = QWidget()
        mainFrame.setLayout(layout)
        self.setCentralWidget(mainFrame)

In [9]:
class MyThread(QThread):
    signalForText = pyqtSignal(str)

    def __init__(self,data=None, parent=None):
        super(MyThread, self).__init__(parent)
        self.data = data

    def write(self, text):
        self.signalForText.emit(str(text))  # 发射信号
    
    def run(self):
        df_result=pd.read_csv('结果.csv')
        df_result=df_result.drop('Unnamed: 0',axis=1).rename(columns={'0':'output'})
        for i in range(0,len(df_result)):
            print(df_result.loc[i,'output'])
            sleep(0.3)
        print("End")   

In [10]:
def main():
    app = QtWidgets.QApplication(sys.argv)
    gui = MainUi()
    main1 = QPixmapDemo()
    main2=QPixmapDemo1()
    
    gui.show()
    gui.search_button_2.clicked.connect(main1.show)
    gui.search_button_3.clicked.connect(main2.show)
    
    df=pd.read_csv('回测指标.csv')
    model = PdTable(df)
    view = QTableView()
    view.setModel(model)
    view.setWindowTitle('回测指标')
    view.resize(600, 800)
    view.setAlternatingRowColors(True)
    gui.search_button_1.clicked.connect(view.show)
  
    sys.exit(app.exec_())

In [11]:
def painting():
    asset_return=(trade_results.pct_change()['trade_price']+1).cumprod()
    all_value_return=(trade_results.pct_change()['all_value']+1).cumprod()
    plt.plot(trade_results.index,asset_return,color = 'red',linewidth=2.5,label='target_asset')
    plt.plot(trade_results.index,all_value_return,color = 'blue',linewidth=2.5,label='asset_value')
    plt.yticks(fontsize=20) 
    plt.xticks(fontsize=20,rotation = 45)
    plt.legend()
    plt.grid(True, color = "lightgrey",axis='y')
    plt.title('asset_value',fontsize = 50,y=1.02)
    #plt.show()
    plt.savefig('回测收益率曲线.jpg')

In [12]:
if __name__ == '__main__':
    #导入策略输出数据
    ##设置输入参数
    result=[]
    start_time='2023-03-14'
    end_time='2023-04-06'
    invest_cash=1000000
    
    trade_data = pd.read_csv('df_beta.csv')
    trade_data=trade_data.set_index('datetime') #将时间作为index，时间需要作为输入变量
    trade_data.index = pd.to_datetime(trade_data.index)
    trade_data=trade_data.loc[start_time:end_time,:]
    
    trade_datelist = trade_data.index.to_list()
    trade_price = trade_data['open'].to_dict() #获取交易价格序列
    trade_signal=trade_data['flag'].to_dict() #获取信号序列
    trade_number=trade_data['quantity'].to_dict() #获取成交量
    trade_volumn=trade_data['quantity'] #输出给orderbook用
    trade_dict = {}
    
    # 调用 Trade 类，进行模拟交易
    trade = Trade(invest_cash)  
    for date in trade_datelist:
        trade.signalcoming(date, price=trade_price[date], signal=trade_signal[date], number=trade_number[date])
        trade_dict[date] = trade.trade()
    
    trade_results = pd.DataFrame.from_dict(trade_dict,'index') # 获得交易持仓净值数据
    trade_results=pd.concat([trade_results,trade_data['quantity']],axis=1)
    trade_results.index = pd.to_datetime(trade_results.index)
    
    df_result=pd.DataFrame(result)
    df_result.to_csv('结果.csv')
    
    df_beta = pd.read_csv('df_beta.csv')
    statistic_beta = pd.DataFrame(cal_statistics(df_beta),index=['斜率方法'])
    statistic_beta.T.to_csv('回测指标.csv')
    
    #painting()#画图
    main()

SystemExit: 0